In [17]:
import numpy as np
import os
from pynwb import NWBFile, NWBHDF5IO, image
from pynwb.ophys import TwoPhotonSeries, OpticalChannel, ImageSegmentation, ImagingPlane
from pynwb.file import Subject
from pynwb.device import Device
import math

import statistics
import pandas as pd
import h5py #Provides methods to supplement NWB
import datajoint as dj #
import csv 
from datetime import datetime
from dateutil import tz
from pynwb import NWBHDF5IO
from matplotlib import pylab as plt

def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)
schema

#Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
schema = dj.schema('MarikeReimer', locals())
schema


#Define Mouse table
@schema
class Mouse(dj.Manual):
    definition = """
    subject_id: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    genotype: enum('B6', 'BalbC', 'Unknown', 'Thy1-YFP')
    sex: enum('M', 'F', 'Unknown')
    species: varchar(128)
    strain: varchar(128)
    """

@schema
class Session(dj.Manual):
    definition = """
    ->Mouse
    identifier: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---' 
    surgery: varchar(128)
    pharmacology: varchar(128)
    """


@schema
class Dendrite(dj.Manual):
    definition = """
    ->Session
    dendrite_id: int                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    soma_center_point: longblob
    proximal_dendrite_length: float
    medial_dendrite_length: float
    distal_dendrite_length: float
    """

@schema
class Image_segmentation(dj.Imported):
    definition = """
    ->Dendrite
    segmentation_name: varchar(128)                  # Primary keys above the '---'
    ---
    #non-primary columns below the '---'
    length: float
    volume: float
    surface_area:float
    spine_type: enum('mushroom', 'thin', 'disconnected', 'stubby', 'U')
    center_of_mass: longblob
    """
    def make(self, key):
        subject_id = key['subject_id']
        identifier = key['identifier']

        nwbfile_to_read = 'C:/Users/meowm/OneDrive/TanLab/DataJointTesting/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb' #TODO: Remove hard coding
        #nwbfile_to_read = 'C:/Users/meowm/Downloads/NWBfiles/' + str(subject_id) + str(identifier) + '.nwb'

        print(nwbfile_to_read)
        with NWBHDF5IO(nwbfile_to_read, 'r') as io:
            nwbfile = io.read()     
            for group in nwbfile.processing["SpineData"]["ImageSegmentation"].children[:]:
                #print(group.name)
                if group.name.startswith("Mushroom"):
                    spine_type = 'mushroom'
                elif group.name.startswith("Thin"):
                    spine_type = 'thin'
                elif group.name.startswith("Disconnected"):
                    spine_type = 'disconnected'
                elif group.name.startswith("Stubby"):
                    spine_type = 'stubby'
                else:
                    spine_type = 'U'

                length = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].length.data[:]
                length = length[0]
                volume = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].volume.data[:]
                volume = volume[0]
                surface_area = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].surface_area.data[:]
                surface_area = surface_area[0]
                center_of_mass = nwbfile.processing["SpineData"]["ImageSegmentation"][group.name].center_of_mass.data[:]
                center_of_mass = center_of_mass[0]
                
                key['segmentation_name'] = group.name 
                key['length'] = length
                key['volume'] = volume
                key['surface_area'] = surface_area
                key['spine_type'] = spine_type
                key['center_of_mass'] = center_of_mass
                self.insert1(key)

@schema
class Distance_to_soma(dj.Computed):
    definition = """
    ->Image_segmentation
    ---
    distance_to_soma: float"""
    def make(self, key):
        center_of_mass = (Image_segmentation() & key).fetch1('center_of_mass')
        soma_center_point = (Dendrite() & key).fetch1('soma_center_point')
        distance_to_soma = math.dist(center_of_mass,soma_center_point)

        key['distance_to_soma'] = distance_to_soma
        self.insert1(key)

#Instantiate tables
mouse = Mouse()
session = Session()
dendrite = Dendrite()
image_segmentation = Image_segmentation()
distance_to_soma = Distance_to_soma()

print(distance_to_soma)

Schema `MarikeReimer`

*subject_id    *identifier    *dendrite_id   *segmentation_ distance_to_so
+------------+ +------------+ +------------+ +------------+ +------------+
L684           Dendrite1a     11             DisconnectedMu 47.4674       
L684           Dendrite1a     11             StubbySpine_0_ 17.7509       
L684           Dendrite1a     11             StubbySpine_10 104.197       
L684           Dendrite1a     11             StubbySpine_11 75.9831       
L684           Dendrite1a     11             StubbySpine_12 75.3099       
L684           Dendrite1a     11             StubbySpine_15 89.2687       
L684           Dendrite1a     11             StubbySpine_18 105.312       
L684           Dendrite1a     11             StubbySpine_1_ 29.4164       
L684           Dendrite1a     11             StubbySpine_2_ 54.7993       
L684           Dendrite1a     11             StubbySpine_3_ 63.3258       
L684           Dendrite1a     11             StubbySpine_4_ 68.2698       
L6

In [ ]:
mouse.drop()

In [ ]:
mouse*session

In [18]:
session & 'subject_id = "L719"'

subject_id Primary keys above the '---',identifier Primary keys above the '---',surgery,pharmacology
L719,Dendrite1,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L719,Dendrite2,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L719,Dendrite3,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L719,Dendrite4,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI
L719,Dendrite5,Contusion,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI


In [11]:
(session & 'subject_id = "L719"' & 'identifier = "Dendrite5"').delete()

[2023-08-11 15:51:03,507][INFO]: Deleting 33 rows from `MarikeReimer`.`__distance_to_soma`
[2023-08-11 15:51:03,956][INFO]: Deleting 33 rows from `MarikeReimer`.`_image_segmentation`
[2023-08-11 15:51:04,394][INFO]: Deleting 1 rows from `MarikeReimer`.`dendrite`
[2023-08-11 15:51:04,809][INFO]: Deleting 1 rows from `MarikeReimer`.`session`
[2023-08-11 15:53:07,849][INFO]: Deletes committed.


1

In [ ]:
session & 'subject_id = "L711"'

In [8]:
(image_segmentation & distance_to_soma & 'subject_id = "L719"' & 'identifier = "Dendrite5"')

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',length,volume,surface_area,spine_type,center_of_mass
L719,Dendrite5,15,MushroomSpine_0_plane_segmentaton,0.534659,0.0795099,57.6362,mushroom,=BLOB=
L719,Dendrite5,15,MushroomSpine_1.001_plane_segmentaton,0.504596,0.0491333,56.1638,mushroom,=BLOB=
L719,Dendrite5,15,MushroomSpine_2.001_plane_segmentaton,1.32095,0.187713,1.0866,mushroom,=BLOB=
L719,Dendrite5,15,StubbySpine_10_plane_segmentaton,0.498863,0.0195887,22.5365,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_11_plane_segmentaton,0.381431,0.0329634,33.9636,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_12_plane_segmentaton,0.414262,0.0563316,44.1899,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_13_plane_segmentaton,0.446357,0.0680903,53.9564,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_14_plane_segmentaton,0.314529,0.0565252,46.4243,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_15_plane_segmentaton,0.32913,0.186374,97.0732,stubby,=BLOB=
L719,Dendrite5,15,StubbySpine_16_plane_segmentaton,0.450846,0.0516417,48.4072,stubby,=BLOB=


In [ ]:
frame = pd.DataFrame(mouse*session*dendrite)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/SubjList.csv")

In [4]:
frame = pd.DataFrame(mouse*session*dendrite*image_segmentation*distance_to_soma)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/UnfilteredDiscData_V1.csv")

In [ ]:
def connect_to_dj():
    dj.config['database.host'] = 'spinup-db001f1f.cluster-cyynsscieqtk.us-east-1.rds.amazonaws.com' 
    dj.config['database.user'] = 'MarikeReimer'
    dj.config['database.password'] = 'uqHKL3YLMCG0'
    dj.conn()

    #Create a schema to organize the pipeline. Defining it here means you only need to change the code in one place.
    current_schema = dj.schema('MarikeReimer', locals())
    return current_schema


schema = connect_to_dj()
print(schema)

In [ ]:
Mouse.drop()

In [ ]:
mouse*session*dendrite*image_segmentation*distance_to_soma

In [2]:
frame = pd.DataFrame(mouse*session*dendrite*(image_segmentation & 'volume >= "0.01"' & 'volume <= "0.8"' &'length >= "0.2"' & 'length <= "3"')*distance_to_soma)
print(frame)
pd.DataFrame(frame).to_csv("C:/Users/meowm/Downloads/DiscSpinedata_V1.csv")

     subject_id  identifier  dendrite_id  \
0          L684  Dendrite1a           11   
1          L684  Dendrite1a           11   
2          L684  Dendrite1a           11   
3          L684  Dendrite1a           11   
4          L684  Dendrite1a           11   
...         ...         ...          ...   
1117       L912   Dendrite3           17   
1118       L912   Dendrite3           17   
1119       L912   Dendrite3           17   
1120       L912   Dendrite3           17   
1121       L912   Dendrite3           17   

                                segmentation_name  genotype sex       species  \
0     DisconnectedMushroomSpine_plane_segmentaton  Thy1-YFP   M  Mus musculus   
1                 StubbySpine_0_plane_segmentaton  Thy1-YFP   M  Mus musculus   
2                StubbySpine_10_plane_segmentaton  Thy1-YFP   M  Mus musculus   
3                 StubbySpine_2_plane_segmentaton  Thy1-YFP   M  Mus musculus   
4                 StubbySpine_3_plane_segmentaton  Thy1-YFP   M  M

In [3]:
# Too small to be spines
mouse*session*dendrite*(image_segmentation & 'volume < "0.01"')*distance_to_soma

subject_id Primary keys above the '---',identifier Primary keys above the '---',dendrite_id Primary keys above the '---',segmentation_name Primary keys above the '---',genotype,sex,species,strain,surgery,pharmacology,soma_center_point,proximal_dendrite_length,medial_dendrite_length,distal_dendrite_length,length,volume,surface_area,spine_type,center_of_mass,distance_to_soma
L684,Dendrite1a,11,StubbySpine_11_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,18.8541,40.9207,0.0,0.108134,0.00708823,0.0845662,stubby,=BLOB=,75.9831
L684,Dendrite1a,11,StubbySpine_12.001_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,18.8541,40.9207,0.0,0.154012,0.00215809,0.0463633,stubby,=BLOB=,75.3099
L684,Dendrite1a,11,ThinSpine_4_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,18.8541,40.9207,0.0,0.211552,0.000801315,0.0349169,thin,=BLOB=,118.774
L684,Dendrite4a,61,StubbySpine_10_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,25.9813,113.845,0.0,0.131208,0.0067187,0.198856,stubby,=BLOB=,20.6327
L684,Dendrite4a,61,StubbySpine_15_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,25.9813,113.845,0.0,0.120722,0.00892247,0.224783,stubby,=BLOB=,20.9353
L684,Dendrite4a,61,StubbySpine_16_plane_segmentaton,Thy1-YFP,M,Mus musculus,C57BL/6,Sham,Romidepsin 2.5mg/kg inject IP 4 weeks after SCI,=BLOB=,25.9813,113.845,0.0,0.152782,0.00688456,0.197962,stubby,=BLOB=,16.3598
L691,Dendrite1a,37,StubbySpine_20_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,16.8129,31.052,0.0,0.173103,0.00756634,0.19607,stubby,=BLOB=,42.5458
L691,Dendrite1a,37,ThinSpine_7_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,16.8129,31.052,0.0,0.260867,0.00151419,0.0896392,thin,=BLOB=,79.286
L691,Dendrite1b,36,StubbySpine_3_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,19.3023,31.0927,0.0,0.152304,0.00515291,0.143759,stubby,=BLOB=,33.2513
L691,Dendrite1b,36,StubbySpine_6_plane_segmentaton,Thy1-YFP,F,Mus musculus,C57BL/6,Sham,Vehicle 1% DMSO inject IP 4 weeks after SCI,=BLOB=,19.3023,31.0927,0.0,0.149885,0.00763356,0.202887,stubby,=BLOB=,42.515
